In [ ]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
import root_pandas as rpd
from root_pandas import read_root

#mpl.rcParams['figure.figsize'] = 6.2,3.5
#mpl.rcParams['axes.labelsize'] = 17
#mpl.rcParams['lines.linewidth'] = 3
#mpl.rcParams['lines.markersize'] = 6
#mpl.rcParams['legend.fontsize']= 15



mpl.rcParams.update({'font.size': 18})
mpl.rcParams.update({'legend.fontsize': 20})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    nbeforecut = inputDataframe.shape[0]
    if nbeforecut>0:
        cutDataframe = inputDataframe.query(cut)
        if text:
            print (text, cutDataframe.shape[0], ' (%2.2f '%(100.0*cutDataframe.shape[0]/nbeforecut), '%)')
    return cutDataframe
def applyHadronCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    dataframe.eval('E = z*Nu', inplace=True)
    dataframe.eval('P = sqrt(E*E-0.1396*0.1396)', inplace=True)
    dataframe.eval('T = E-0.1396', inplace=True)
    dataframe.eval('pt2 = pt*pt', inplace=True)

    #dataframe = applyCut(dataframe, 'Q2>1.0 and Q2<1.25', ' 1.0 <Q2<1.25 :')
    dataframe = applyCut(dataframe, 'Nu>2.2 and Nu<4.25',  ' 2.2 <Nu<4.25')
    dataframe = applyCut(dataframe, 'P <2.5 ', 'P<2.5 ')
    dataframe = applyCut(dataframe, 'P >0.2 ', 'P>0.2 ')

    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    #dataframe = applyCut(dataframe,'(pid==211 & theta_lab_asso>10.0)|(pid_asso==-211 & theta_lab_asso>30 & theta_lab_asso<120)')
    #dataframe = applyCut(dataframe, '(pid==211 & P_asso>0.200) |(pid_asso==-211 & P_asso>0.200) ', 'pi+ P_asso > 200 MeV, pi- P_asso > 300 MeV')
    return dataframe

def applyElectronCuts(fullDataframe,name='default',isMC=False): 
    dataframe = fullDataframe
    print ('Entries before cut ', dataframe.shape[0])
    dataframe.eval('inelasticity = Nu/5.014', inplace=True)
    #dataframe = applyCut(dataframe, 'Q2>1.0 and Q2<1.25', ' 1.0 <Q2<1.25 :')
    dataframe = applyCut(dataframe, 'Nu>2.2 and Nu<4.25',  ' 2.2 <Nu<4.25')
    dataframe = applyCut(dataframe, 'inelasticity<0.85','inelasticity < 0.85')
    return dataframe



#this needs to be read in chunks otherwise it uses all memory. (from https://github.com/scikit-hep/root_pandas)
def getdatainChunks(filename,treename):
    dataframe =pd.DataFrame()
    for df in read_root(filename, treename, chunksize=100000) :
        #print df.shape[0]
        dataframe = pd.concat([dataframe,df])
    
    print (dataframe.shape[0])
    return dataframe
    

In [ ]:
df = {}

In [ ]:
for target in ['D','C','Fe','Pb']:
    #GiBUU
    print (target)
    print (' hadron')
    df['GiBUU_%s_hadron'%target]= getdatainChunks('GiBUU_SingleHadron_%s.root'%target, '%s_hadron'%target)
    print (' electron')
    df['GiBUU_%s_electron'%target]= getdatainChunks('GiBUU_SingleHadron_%s.root'%target,'%s_electron'%target)
print (df.keys())

In [ ]:
print (df.keys())

In [ ]:
for target in ['D','C','Fe','Pb']:
    print (target)
    df['GiBUU_%s_electron'%target] = applyElectronCuts(df['GiBUU_%s_electron'%target], isMC=True)
    df['GiBUU_%s_hadron'%target] = applyHadronCuts(df['GiBUU_%s_hadron'%target], isMC=True)
    print (' ') 


In [ ]:
df['GiBUU_D_hadron'].hist(figsize=(15, 15),density=True,alpha=0.5,bins=100)
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(14,8),sharex=True, sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

import matplotlib.colors as mcolors

WEdges = np.linspace(2.2,2.8,6)
Q2Edges = [1.0,1.4,1.6,2.0,4.1]

print(WEdges)
Wbins = [(x, y) for (x, y) in zip(WEdges[:-1], WEdges[1:])]
Q2bins = [(x, y) for (x, y) in zip(Q2Edges[:-1], Q2Edges[1:])]

df['GiBUU_D_hadron'].eval('pt2 = pt*pt',inplace=True)
df['GiBUU_Pb_hadron'].eval('pt2 = pt*pt',inplace=True)

for iq2, q2bin in enumerate(Q2bins):
    print(iq2)
    for i, ibin in enumerate(Wbins):
        row = 0 
        if(i>4): row = 1
        col = i
        if col>4: col = col-4
        #print(row,col)
        query = 'pid==211 and W>%2.2f and W<%2.2f and Q2>%2.2f and Q2<%2.2f'%(ibin[0],ibin[1],q2bin[0],q2bin[1])
        #print(query)
        datacut= df['GiBUU_D_hadron'].query(query)
        axs[0,col].set_title(' W>%2.2f and W<%2.2f'%(ibin[0],ibin[1]))
        axs[iq2,0].yaxis.set_label_text('%2.2f<Q2<%2.2f'%(q2bin[0],q2bin[1]), fontsize=13)

        axs[iq2,col].hist2d(datacut['z'],datacut['pt2'],range=([0,1],[0,2.0]), norm=mcolors.LogNorm(), bins=50)
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 5, figsize=(14,8),sharex=True, sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})

import matplotlib.colors as mcolors

WEdges = np.linspace(2.2,2.8,6)
Q2Edges = [1.0,1.4,1.6,2.0,4.1]

print(WEdges)
Wbins = [(x, y) for (x, y) in zip(WEdges[:-1], WEdges[1:])]
Q2bins = [(x, y) for (x, y) in zip(Q2Edges[:-1], Q2Edges[1:])]

df['GiBUU_D_hadron'].eval('pt2 = pt*pt',inplace=True)
df['GiBUU_Pb_hadron'].eval('pt2 = pt*pt',inplace=True)

for iq2, q2bin in enumerate(Q2bins):
    print(iq2)
    for i, ibin in enumerate(Wbins):
        row = 0 
        if(i>4): row = 1
        col = i
        if col>4: col = col-4
        #print(row,col)
        query = 'pid==211 and W>%2.2f and W<%2.2f and Q2>%2.2f and Q2<%2.2f'%(ibin[0],ibin[1],q2bin[0],q2bin[1])
        #print(query)
        datacut= df['GiBUU_Pb_hadron'].query(query)
        axs[0,col].set_title(' W>%2.2f and W<%2.2f'%(ibin[0],ibin[1]))
        axs[iq2,0].yaxis.set_label_text('%2.2f<Q2<%2.2f'%(q2bin[0],q2bin[1]), fontsize=13)

        axs[iq2,col].hist2d(datacut['z'],datacut['pt2'],range=([0,1],[0,2.0]), norm=mcolors.LogNorm(), bins=50)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,6),sharex=True, sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})
axs[0].hist2d(df['GiBUU_D_hadron'].query('pid==211')['z'],df['GiBUU_D_hadron'].query('pid==211')['pt2'],range=([0,1.0],[0,2.0]), norm=mcolors.LogNorm(), bins=50)
axs[1].hist2d(df['GiBUU_Pb_hadron'].query('pid==211')['z'],df['GiBUU_Pb_hadron'].query('pid==211')['pt2'],range=([0,1.0],[0,2.0]), norm=mcolors.LogNorm(), bins=50)
axs[0].set_title('GiBUU: Deuterium',fontsize=20)
axs[1].set_title('GiBUU: Lead',fontsize=20)
axs[0].yaxis.set_label_text('pt2',fontsize=20)
axs[1].xaxis.set_label_text('z',fontsize=20)
axs[0].xaxis.set_label_text('z',fontsize=20)

plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,6),sharex=True, sharey=True,gridspec_kw={'hspace': 0, 'wspace':0})
axs[0].hist2d(df['GiBUU_D_hadron'].query('pid==211')['z'],df['GiBUU_D_hadron'].query('pid==211')['MissingMass'],range=([0.0,1.0],[-1.0,2.2]), norm=mcolors.LogNorm(), bins=50)
axs[1].hist2d(df['GiBUU_Pb_hadron'].query('pid==211')['z'],df['GiBUU_Pb_hadron'].query('pid==211')['MissingMass'],range=([0.0,1.0],[-1.0,2.2]), norm=mcolors.LogNorm(), bins=50)
axs[0].set_title('GiBUU: Deuterium',fontsize=20)
axs[1].set_title('GiBUU: Lead',fontsize=20)
axs[0].yaxis.set_label_text('pt2',fontsize=20)
axs[1].xaxis.set_label_text('z',fontsize=20)
axs[0].xaxis.set_label_text('z',fontsize=20)

plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.hist(df['GiBUU_D_hadron'].query('pid==211')['MissingMass'], density=True, range=(0,3.0),bins=100,alpha=0.3,label='GiBUU D')
plt.hist(df['GiBUU_C_hadron'].query('pid==211')['MissingMass'], density=True,range=(0.0,3.0), bins=100,histtype='step',label='GiBUU C')
plt.hist(df['GiBUU_Fe_hadron'].query('pid==211')['MissingMass'], density=True,range=(0.0,3.0), bins=100,histtype='step',label='GiBUU Fe')
plt.hist(df['GiBUU_Pb_hadron'].query('pid==211')['MissingMass'], density=True,range=(0.0,3.0), bins=100,histtype='step',label='GiBUU Pb')
#plt.yscale('log')
plt.ylabel('Normalized entries',fontsize=22)
plt.xlabel('Missing Mass',fontsize=22)
plt.title(r'$e\pi^{+}+X$ final state',fontsize=22)
plt.legend(loc='best',fontsize=22,frameon=False)
plt.show()
